In [2]:
from SPM import subspace_power_method
import numpy as np
from itertools import permutations
from scipy.optimize import minimize
from scipy.stats import uniform_direction
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import helper_functions
%matplotlib inline

In [3]:
def recovermixingmatrix(I,J,A):
    # generate the second and fourth cumulant tensors
    # generate a random mixing matrix with columns having norm 1
    second_order_kstats = A@ np.transpose(A)
    # fourth cumulant tensor
    fourth_order_kstats=helper_functions.generate_lowrank_tensor(A[:,:-1],4)
    # recover first J-1 columns via tensor decomposition
    cols,lambdas=subspace_power_method(fourth_order_kstats,n=4,d=I,r=J-1)
    def returnmindistancebewteenvectors(cols):
        def distancebewteenvectors(v1,v2):
            v1=v1.reshape(-1,1)
            v2=v2.reshape(-1,1)
            M=v1@np.transpose(v1)-v2@np.transpose(v2)
            return np.sum(M*M)
        lens=cols.shape[1]
        error=1
        for i in range(lens):
            for j in range(i+1,lens):
                error=min(error,distancebewteenvectors(cols[:,i],cols[:,j]))
        return error
    step=0
    while returnmindistancebewteenvectors(cols)<0.1 and step<100:
        cols,lambdas=subspace_power_method(fourth_order_kstats,n=4,d=I,r=J-1)
        step+=1
    
    # now we use second cumulant to find the last column of the mixing matrix 
    # by minimizing the distance bewteen a matrix in the linear span of the second cumulant matrix 
    # and all the rank 1 matrices M_1,...M_{J-1} obtained from the first J-1 columns of A
    rank1_matrixlist=[]
    for i in range(J-1):
        vector=cols[:,i].reshape(-1,1)
        matrix=vector@ np.transpose(vector)
        rank1_matrixlist.append(matrix)
    
    #function returning the distance between second_order_kstats-l_1*M_1-...-l_{J-1}*M_{J-1} and v\otimes v
    # input is the vector (l|v)
    def sumofsquare(x):
        l=x[:J]
        v=x[J:]
        v=v.reshape(-1,1)
        n=len(l)
        M=second_order_kstats
        for i in range(n-1):
            M=M-l[i]*rank1_matrixlist[i]
        M=M-l[-1]*v@np.transpose(v)
        return np.sum(M*M)
    #initialize a random start point
    l0= np.random.randn(J).reshape(-1,1)
    v0 = np.transpose(uniform_direction(I).rvs(1))
    x=np.vstack((l0,v0)).reshape(-1)
    # optimizition
    es = minimize(sumofsquare, x, method='Powell')
    # recover last column of A
    v=es.x[J:]/np.linalg.norm(es.x[J:])
    #recover A up to permutation and sign
    estimateA=np.hstack((cols,v.reshape(-1,1)))
    return estimateA

In [9]:
# function testing how well is the approximation
def similarity_measures(B,A):
    J=A.shape[1]
    columnlist=[]
    for i in range(J):
        v=A[:,i]
        dislist=[]
        signlist=[]
        for j in range(B.shape[1]):
            u=abs(v-B[:,j])
            uprime=abs(v+B[:,j])
            sign=1
            if np.sum(u)<np.sum(uprime):
                dislist.append(np.sum(u))
                signlist.append(sign)
            else:
                sign=-1
                dislist.append(np.sum(uprime))
                signlist.append(sign)
        a=min(dislist)
        index=dislist.index(a)
        sign=signlist[index]
        columnlist.append(sign*B[:,index])
        B=np.delete(B,index,1)
    newAes=np.transpose(np.vstack(tuple(columnlist)))
    C= newAes-A
    relfroberror=(np.sum(C*C)/J)**0.5
    froberror=(np.sum(C*C))**0.5
    cosine_similarity=np.mean(np.sum(newAes*A,axis=0))
    last_column_cosine_similarity=np.sum(newAes[:,-1]*A[:,-1])
    # for now the bound is 0.95
    if last_column_cosine_similarity>=0.95:
        succesfulflag_last_column=1
    else:
        succesfulflag_last_column=0
    return last_column_cosine_similarity,cosine_similarity,relfroberror,froberror,succesfulflag_last_column,newAes,A


#apply algorithm
def algorithm(I,J):
    uniformsphere=uniform_direction(I)
    A=np.transpose(uniformsphere.rvs(J))
    estimateA=recovermixingmatrix(I,J,A)
    return similarity_measures(estimateA,A)

def algorithm_allJ(I,Jlist):
    last_row_cosine_similaritylist=[]
    relative_frobneuserror_list=[]
    frobneuserror_list=[]
    successful_list=[]
    for J in Jlist:
        (last_column_cosine_similarity,cosine_similarity,relfroberror,froberror,succesfulflag_last_column,newAes,A)=algorithm(I,J)
        last_row_cosine_similaritylist.append(last_column_cosine_similarity)
        relative_frobneuserror_list.append(relfroberror)
        frobneuserror_list.append(froberror)
        successful_list.append(succesfulflag_last_column)
    return last_row_cosine_similaritylist,relative_frobneuserror_list,frobneuserror_list,successful_list




In [42]:
Iequals4Jlist=list(range(2,13))
Iequals5Jlist=list(range(2,18))
Iequals6Jlist=list(range(2,24))
Iequals7list=list(range(2,31))
Iequals8list=list(range(2,39))
Iequals9list=list(range(2,48))

def return_mean_relfrob(I,Jlist,n_iter,n_jobs):
    output=Parallel(n_jobs=n_jobs)(delayed(algorithm_allJ)(I,J) for I,J in [(I,Jlist)]*n_iter)
    last_column_cosine_similaritylistlist,relfroberrorlistlist,froberrorlistlist,succesfulflag_last_columnlistlistlist=zip(*output)
    last_row_cosine_similarityarry=np.array(last_column_cosine_similaritylistlist)
    relative_frobneuserror_listarray=np.array(relfroberrorlistlist)
    frobneuserror_listarray=np.array(froberrorlistlist)
    successful_listarray=np.array(succesfulflag_last_columnlistlistlist)

    rel_frob_mean=np.mean(relative_frobneuserror_listarray,axis=0)
    return rel_frob_mean


def plot_one_graph_6789(n_iter,n_jobs):
    Iequals6Jlist=list(range(2,24))
    Iequals7Jlist=list(range(2,31))
    Iequals8Jlist=list(range(2,39))
    Iequals9Jlist=list(range(2,48))
    relfrob_mean_6=return_mean_relfrob(6,Iequals6Jlist,n_iter,n_jobs)
    relfrob_mean_7=return_mean_relfrob(7,Iequals7Jlist,n_iter,n_jobs)
    relfrob_mean_8=return_mean_relfrob(8,Iequals8Jlist,n_iter,n_jobs)
    relfrob_mean_9=return_mean_relfrob(9,Iequals9Jlist,n_iter,n_jobs)
    fig,((ax1,ax2),(ax3,ax4),(ax5,ax6))=plt.subplots(3,2,figsize=(36,36))
    ax1.plot(Iequals6Jlist,relfrob_mean_6)
    ax1.set_title('I=6}')
    ax2.plot(Iequals7Jlist,relfrob_mean_7)
    ax2.set_title('I=7}')
    ax3.plot(Iequals8Jlist,relfrob_mean_8)
    ax3.set_title('I=8}')
    ax4.plot(Iequals9Jlist,relfrob_mean_9)
    ax4.set_title('I=9}')
    ax1.set_xticks(Iequals6Jlist)
    ax2.set_xticks(Iequals7Jlist)
    ax3.set_xticks(Iequals8Jlist)
    ax4.set_xticks(Iequals9Jlist)
    ax1.axvline(x=15,color='red',linestyle='-')
    ax2.axvline(x=21,color='red',linestyle='-')
    ax3.axvline(x=28,color='red',linestyle='-')
    ax4.axvline(x=36,color='red',linestyle='-')

    ax5.plot(Iequals6Jlist,relfrob_mean_6,label='I=6')
    ax5.plot(Iequals7Jlist,relfrob_mean_7,label='I=7')
    ax5.plot(Iequals8Jlist,relfrob_mean_8,label='I=8')
    ax5.plot(Iequals8Jlist,relfrob_mean_8,label='I=9')
    ax5.axvline(x=15,color='black',linestyle='--')
    ax5.axvline(x=21,color='black',linestyle='--')
    ax5.axvline(x=28,color='black',linestyle='--')
    ax5.axvline(x=36,color='black',linestyle='--')
    ax5.legend()

    ax6.plot(Iequals6Jlist,relfrob_mean_6,label='I=6')
    ax6.plot(np.array(Iequals7Jlist)*(5/7),relfrob_mean_7,label='I=7')
    ax6.plot(np.array(Iequals8Jlist)*(15/28),relfrob_mean_8,label='I=8')
    ax6.plot(np.array(Iequals8Jlist)*(15/36),relfrob_mean_8,label='I=9')
    ax6.legend()
    ax6.axvline(x=15,linestyle='--',color='black')
    plt.savefig(f'accurate_tensor_plot,I=6,7,8,9,n_iter={n_iter}')


In [ ]:
plot_one_graph_6789(1000,20)